# VARNN Training Codebase To Share

**Last Updated 6/21/2022**

In [ ]:
!pip install statsmodels==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 8.0 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tqdm.notebook import tqdm, trange
import copy
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import itertools
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os
import sys
from functools import reduce
from itertools import product
from sklearn.linear_model import LinearRegression
import random
from datetime import datetime
from statsmodels.tsa.api import VAR

os.chdir('drive/MyDrive/EconML DL/')
sys.path.insert(0, 'Shared')

from varnn_training import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_dir = 'drive/MyDrive/EconML DL'

### Loading Data

In [ ]:
# Loading the Data - 7-variables Monthly

data = pd.read_csv('Forecasting/monthlyData.csv')
x_d_all = data[['L0_OILPRICEx', 'L0_EXUSUKx', 'L0_S.P.500', 'L0_TB3MS', 'L_0y', 'L0_UNRATE', 'L0_HOUST']]
x_d_all['L0_HOUST'] = x_d_all['L0_HOUST'].diff()
x_d_all = x_d_all.dropna()
x_d_all.columns = ['oil', 'Ex', 'SPY', 'DGS3', 'inf', 'unrate', 'house_starts']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
data = pd.read_csv('VARCTIC8.csv')
data = data.dropna()
x_d_all = data[['CO2_MaunaLoa', 'TCC', 'PR', 'AT', 'SST', 'SIE', 'SIT', 'Albedo']]
x_d_all.columns = ['CO2_MaunaLoa', 'TCC', 'PR', 'AT', 'SST', 'SIE', 'SIT', 'Albedo']
x_d_all_colnames = x_d_all.columns
n_var = len(x_d_all.columns)
var_names = list(x_d_all_colnames)

In [ ]:
# Load Simulation Data
out = np.load(f'Simulation Data/july_simulated_data_group1.npz', allow_pickle = True)
sim_data_all = out['sim_data_all']
exog_all = out['exog_all']
#sim_data = sim_data_all[:,:,0]
#exog = exog_all[:, :,0]

### Execution Code

**Define Experiment Parameters**

In [ ]:
# PARAMETERS
num_inner_bootstraps = 100
num_repeats = 2

nn_hyps = {
    # Mostly unchanged hyperparameters
    'epochs': 500,
    'show_train': 3,
    'opt_bootstrap': 2,
    'num_bootstrap': num_inner_bootstraps, 
    'sampling_rate': 0.75,
    'block_size': 12,
    'cancel_out': False,
    'standardize': True,
    'prior_shift': False,
    'prior_shift_strength': 0, 
    'oob_loss_multiple_threshold': 5,
    'save_models': False,
    'exog': None,

    # Hyperparamters of interest - but not changed in this experiment
    'nodes': [200, 200, 200],
    'tvpl_archi': [50, 50],
    'patience': 50,
    'tol': 0.0001,
    'lr': 0.001,
    'lr_multiple': 0.9975,
    'marx': True,
    'dummy_interval': 6,
    'l1_input_lambda': 0,
    'l0_input_lambda': 0,
    'precision_lambda': 0,
    'input_dropout_rate': 0,
    'vsn': False, 
    'fcn': False, 
    'eqn_by_eqn': False,
    'time_hemi_prior_variance': 1,
    'fix_bootstrap': False,
    'loss_weight_param': 0.5,
    'log_det_multiple': 1,
    'optimizer': 'Adam',
    'n_lag_d': 12, 'n_lag_linear': 6, 'n_lag_ps': 6,
    'variables': ['oil', 'Ex', 'SPY', 'DGS3', 'inf', 'unrate', 'house_starts']
}

In [ ]:
# # OPTION 1: Convenient creation of lists of experiment params

# baseline_model = {'nodes': [100, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [50], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam', 'n_exog': 2,
#                       's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(37)), list(range(37, 203))], 'joint_estimation': True, 'name': '2 Exog'}



# # Dictionary of param name and list of values to try
# params_to_vary = {
#                   'precision_lambda': [0, 0.5, 1],
#                   'lambda_temper_epochs': [20, 40],
#                   'joint_estimation': [False],
#                   'nodes': [[100], [100, 100, 100, 100], [400, 400]],
#                   'dropout_rate': [0, 0.5, 0.75],
#                   'time_dummy_setting': [1],
#                   'actv': ['nn.SELU()'], 
# }

# ### Create list of models to try (Experiment Params) - DONE
# experiment_params = [baseline_model]

# experiment_params_add = [{'nodes': [100, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [50], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam', 'n_exog': 1,
#                       's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(27, 32)), list(range(32, 198))], 'joint_estimation': True, 'name': 'Only Exog'},
#                       {'nodes': [100, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [50], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam', 'n_exog': 1,
#                       's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(32)), list(range(32, 198))], 'joint_estimation': True, 'name': 'Normal Endog'},
#                       {'nodes': [100, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [50], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam', 'n_exog': 5,
#                       's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(52)), list(range(52, 218))], 'joint_estimation': True, 'name': '5 Exog'}
# ]

# experiment_params += experiment_params_add

# experiment_results = {}

# for param, param_values in params_to_vary.items():
#   for param_value in param_values:
#     baseline_model_copy = baseline_model.copy()
#     baseline_model_copy.update({param: param_value,'name': f'{param}: {param_value}'})
#     experiment_params.append(baseline_model_copy)

In [ ]:
experiment_params = [
                     {'nodes': [400, 200, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [5], 'lr': 0.002, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': True, 'name': 'RELU TV5 High LR'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [5], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': True, 'name': 'RELU TV5'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.SELU()', 'tvpl_archi': [5], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': True, 'name': 'SELU TV5'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.SELU()', 'tvpl_archi': [5], 'lr': 0.0002, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': True, 'name': 'SELU TV5 Low LR'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.SELU()', 'tvpl_archi': [5], 'lr': 0.0005, 'time_dummy_setting': 0, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 174))], 'joint_estimation': True, 'name': 'RELU TV5 Trends'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.ReLU()', 'tvpl_archi': [5], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': False, 'name': 'Joint Off RELU'},
                     {'nodes': [400, 200, 100], 'actv': 'nn.SELU()', 'tvpl_archi': [5], 'lr': 0.0005, 'time_dummy_setting': 2, 'dropout_rate': 0.25, 'precision_lambda': 0.25, 'lambda_temper_epochs': 50, 'optimizer': 'Adam',
                      's_pos_setting': {'is_hemi': False, 'n_times': 30}, 's_pos': [list(range(144)), list(range(144, 226))], 'joint_estimation': False, 'name': 'Joint Off SELU'},
]

In [ ]:
# @title Get Bootstrap Indices

# Function to generate the same bootstrap indices, so they can be constant across the different experiments for hyp tuning

def get_bootstrap_indices(num_bootstrap, n_obs, block_size, sampling_rate, opt_bootstrap):

  bootstrap_indices = []

  for j in range(num_bootstrap):
    if opt_bootstrap == 1:
      # Sample the bootstrap indices
      k = int(sampling_rate * n_obs)

      in_sample = sorted(random.sample(list(range(n_obs)), k = k))
      oob = [e for e in list(range(X_train.shape[0])) if e not in in_sample]

    if opt_bootstrap == 2: # Block bootstrap
      # Select the size of first block
      first_block_size = random.sample(list(range(int(block_size / 2), block_size + 1)), k = 1)[0]
      # Get the starting ids of the blocks
      block_start_ids = [0] + list(range(first_block_size, n_obs, block_size))

      # If last block size < half of block size
      last_block_size = n_obs - block_start_ids[-1]
      if last_block_size < block_size / 2:
        block_start_ids.remove(block_start_ids[-1])

      num_oob_blocks = int(((1-sampling_rate) * n_obs) / block_size)
      oob_blocks = random.sample(list(range(len(block_start_ids))), k = num_oob_blocks)
      # Get the OOB indices
      oob = list(itertools.chain(*[list(range(block_start_ids[e], block_start_ids[e+1])) if e < len(block_start_ids) - 1 else list(range(block_start_ids[e], n_obs)) 
        for e in oob_blocks]))
      
      in_sample = [e for e in list(range(n_obs)) if e not in oob]

    bootstrap_indices.append({'in_sample': in_sample, 'oob': oob})
  return bootstrap_indices

In [ ]:
# Specify Lists of #Variables you want the experiment to run over
variable_lists = [
            #     ['inf', 'unrate', 'DGS3']
                  ['CO2_MaunaLoa', 'TCC', 'PR', 'AT', 'SST', 'SIE', 'SIT', 'Albedo']
]

### Execution

In [ ]:
experiment_name = '7jul_varctic'
test_size = 100

folder_path = f'New Experiments/{experiment_name}'
if os.path.isdir(folder_path) == False:
  os.mkdir(folder_path)
else:
  print('Folder already exists')

for variable_list in variable_lists:

  print('Variables', variable_list)
  x_d = x_d_all.copy()
  x_d = x_d[variable_list]
  x_d_colnames = x_d.columns
  var_names = x_d.columns
  n_var = len(var_names)
  nn_hyps['variable_list'] = variable_list

  train_size = x_d.shape[0] - test_size - nn_hyps['n_lag_d']
  if nn_hyps['fix_bootstrap'] == True:
    bootstrap_indices = get_bootstrap_indices(num_bootstrap = num_inner_bootstraps, n_obs = train_size, block_size = nn_hyps['block_size'], sampling_rate = nn_hyps['sampling_rate'], opt_bootstrap = nn_hyps['opt_bootstrap'])
    nn_hyps['bootstrap_indices'] = bootstrap_indices
  else:
    nn_hyps['bootstrap_indices'] = None

  # Save experiment param list
  with open(f'{folder_path}/params.npz', 'wb') as f:
      np.savez(f, params = experiment_params, date = datetime.now())

  for repeat in range(num_repeats):
    for i in range(len(experiment_params)):

      nn_hyps.update(experiment_params[i])

      n_betas = n_var * nn_hyps['n_lag_linear'] + 1
      n_inputs_wo_time = n_var * (nn_hyps['n_lag_linear'] + nn_hyps['n_lag_d'])

      X_train, X_test, Y_train, Y_test, _, _, nn_hyps = process_varnn_data(x_d, nn_hyps, test_size = test_size, n_time_trends = 100, time_dummy_setting = nn_hyps['time_dummy_setting'], marx = nn_hyps['marx'], dummy_interval = nn_hyps['dummy_interval'])

      n_inputs_total = X_train.shape[1]
      nn_hyps['neurons_weights'] = [nn_hyps['tvpl_archi'] for i in range(n_betas)]

      # If s_pos is already not defined (s_pos can be defined by user)
      if not nn_hyps.get('s_pos'):
        # s_pos_setting
        if nn_hyps['s_pos_setting']['is_hemi'] == False:
          nn_hyps['s_pos'] = [ list(range(n_inputs_total)) ]
        else:
          n_inputs_total_new = n_inputs_wo_time + nn_hyps['s_pos_setting']['n_times']
          nn_hyps['s_pos'] = [ list(range(n_inputs_wo_time)), list(range(n_inputs_wo_time, n_inputs_total_new))]
          # Subset the X_train and X_test to only the required columns
          X_train = X_train[:, :n_inputs_total_new]
          X_test = X_test[:, :n_inputs_total_new]

      print(nn_hyps['s_pos'])
    
      results = conduct_bootstrap(X_train, X_test, Y_train, Y_test, nn_hyps, device)

      BETAS = results['betas_draws']
      BETAS_IN = results['betas_in_draws']
      SIGMAS = results['sigmas_draws']
      SIGMAS_IN = results['sigmas_in_draws']
      PRECISION = results['precision_draws']
      PRECISION_IN = results['precision_in_draws']
      CHOLESKY = results['cholesky_draws']
      CHOLESKY_IN = results['cholesky_in_draws']
      PREDS = results['pred_in_ensemble'] 
      PREDS_TEST = results['pred_ensemble']

      with open(f'{folder_path}/params_{i}_repeat_{repeat}.npz', 'wb') as f:
        np.savez(f, betas = BETAS, betas_in = BETAS_IN, 
                sigmas = SIGMAS, sigmas_in = SIGMAS_IN,
                precision = PRECISION, precision_in = PRECISION_IN,
                cholesky = CHOLESKY, cholesky_in = CHOLESKY_IN,
                train_preds = PREDS, test_preds = PREDS_TEST, 
                y = Y_train, y_test = Y_test, 
                params = nn_hyps)
      
      del nn_hyps['s_pos']

Output hidden; open in https://colab.research.google.com to view.